In [1]:
!pip install psycopg2

In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
import pandas.io.sql as sqlio
from sqlalchemy import create_engine

In [2]:
# Read in csv files
dce_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_dce.csv")
#fcev_df=pd.read_csv("..Data/NREL 2019 Survey Data/data/survey_res_fcev.csv")
main_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_main.csv")
person_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_person.csv")
pev_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_pev.csv")
vehicle_df=pd.read_csv("../Data//NREL 2019 Survey Data/data/survey_res_vehicle.csv")

## OVERVIEW OF DATA TABLES

In [3]:
# Discrete Choice Experiment table
print(dce_df.shape)
dce_df.head()

(33984, 89)


,sampno,sequence,alt_order,flag_dce_year,alt_1_vehtype,alt_2_vehtype,alt_3_vehtype,alt_4_vehtype,alt_1_fueltype,alt_2_fueltype,...,alt_4_purchincent,alt_1_anncost,alt_2_anncost,alt_3_anncost,alt_4_anncost,alt_1_accel,alt_2_accel,alt_3_accel,alt_4_accel,choice
0,190329,1,"3,1,2,4",2025,8,11,12,3,1,3,...,1,1660,747,996,646,8.5,5.2,5.9,8.9,1
1,190329,2,"3,1,2,4",2025,8,6,8,11,3,1,...,1,1494,799,2075,830,8.1,8.2,8.5,5.5,1
2,190329,3,"3,1,2,4",2025,8,8,9,2,2,8,...,1,2490,1660,1081,1153,7.5,6.8,5.9,8.3,2
3,190329,4,"3,1,2,4",2025,8,8,10,11,8,7,...,1,1660,1037,403,830,10.2,6.5,7.9,5.5,1
4,190329,5,"3,1,2,4",2025,4,4,7,11,2,7,...,1,1844,1153,2213,692,5.1,6.6,6.5,5.5,1


In [4]:
print(dce_df.dtypes)
dce_df.nunique()

sampno             int64
sequence           int64
alt_order         object
flag_dce_year      int64
alt_1_vehtype      int64
                  ...   
alt_1_accel      float64
alt_2_accel      float64
alt_3_accel      float64
alt_4_accel      float64
choice             int64
Length: 89, dtype: object


sampno           4248
sequence            8
alt_order          24
flag_dce_year       2
alt_1_vehtype      13
                 ... 
alt_1_accel        98
alt_2_accel        98
alt_3_accel        98
alt_4_accel        98
choice              4
Length: 89, dtype: int64

In [5]:
dce_df.count()

sampno           33984
sequence         33984
alt_order        33984
flag_dce_year    33984
alt_1_vehtype    33984
                 ...  
alt_1_accel      33984
alt_2_accel      33984
alt_3_accel      33984
alt_4_accel      33984
choice           33984
Length: 89, dtype: int64

In [6]:
#print(fcev_df.shape)
#fcev_df.head()

In [7]:
#print(fcev_df.dtypes)
#print(fcev_df.nunique())

In [8]:
# Main demographics table
print(main_df.shape)
main_df.head()

(4248, 304)


,sampno,source,language,age_grp,california,county,region,future_decision_role,num_hh_vehicles,household_members_1,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,192850,4,1,3,1,37,4,1,2,0,...,2,2,3,3,1,1,,2,1,11
1,193449,4,1,3,1,34,6,3,3,0,...,3,2,2,2,3,1,,1,,8
2,193440,4,1,3,1,50,1,1,2,0,...,4,2,3,2,1,1,,2,2,11
3,193133,4,1,4,1,33,2,1,1,0,...,2,2,1,2,3,1,,2,2,7
4,193400,4,1,3,1,1,5,3,3,0,...,4,2,2,2,2,1,,2,2,7


In [9]:
# Create list of columns to drop for ML model
col_drop=[
    'vehicletype_consideration_1',
    'vehicletype_consideration_2',
    'vehicletype_consideration_3',
    'vehicletype_consideration_4',
    'vehicletype_consideration_5',
    'vehicletype_consideration_6',
    'vehicletype_consideration_7',
    'vehicletype_consideration_8',
    'vehicletype_consideration_9',
    'vehicletype_consideration_10',
    'vehicletype_consideration_11',
    'vehicletype_consideration_12',
    'vehicletype_consideration_13',
    'fueltype_consideration_1_1',
    'fueltype_consideration_2_1',
    'fueltype_consideration_3_1',
    'fueltype_consideration_4_1',
    'fueltype_consideration_5_1',
    'fueltype_consideration_6_1',
    'fueltype_consideration_7_1',
    'fueltype_consideration_8_1',
    'fueltype_consideration_9_1',
    'fueltype_consideration_10_1',
    'fueltype_consideration_11_1',
    'fueltype_consideration_12_1',
    'fueltype_consideration_13_1',
    'fueltype_consideration_1_2',
    'fueltype_consideration_2_2',
    'fueltype_consideration_3_2',
    'fueltype_consideration_4_2',
    'fueltype_consideration_5_2',
    'fueltype_consideration_6_2',
    'fueltype_consideration_7_2',
    'fueltype_consideration_8_2',
    'fueltype_consideration_9_2',
    'fueltype_consideration_10_2',
    'fueltype_consideration_11_2',
    'fueltype_consideration_12_2',
    'fueltype_consideration_13_2',
    'fueltype_consideration_1_3',
    'fueltype_consideration_2_3',
    'fueltype_consideration_3_3',
    'fueltype_consideration_4_3',
    'fueltype_consideration_5_3',
    'fueltype_consideration_6_3',
    'fueltype_consideration_7_3',
    'fueltype_consideration_8_3',
    'fueltype_consideration_9_3',
    'fueltype_consideration_10_3',
    'fueltype_consideration_11_3',
    'fueltype_consideration_12_3',
    'fueltype_consideration_13_3',
    'fueltype_consideration_1_4',
    'fueltype_consideration_2_4',
    'fueltype_consideration_3_4',
    'fueltype_consideration_4_4',
    'fueltype_consideration_5_4',
    'fueltype_consideration_6_4',
    'fueltype_consideration_7_4',
    'fueltype_consideration_8_4',
    'fueltype_consideration_9_4',
    'fueltype_consideration_10_4',
    'fueltype_consideration_11_4',
    'fueltype_consideration_12_4',
    'fueltype_consideration_13_4',
    'fueltype_consideration_1_5',
    'fueltype_consideration_2_5',
    'fueltype_consideration_3_5',
    'fueltype_consideration_4_5',
    'fueltype_consideration_5_5',
    'fueltype_consideration_6_5',
    'fueltype_consideration_7_5',
    'fueltype_consideration_8_5',
    'fueltype_consideration_9_5',
    'fueltype_consideration_10_5',
    'fueltype_consideration_11_5',
    'fueltype_consideration_12_5',
    'fueltype_consideration_13_5',
    'fueltype_consideration_1_6',
    'fueltype_consideration_2_6',
    'fueltype_consideration_3_6',
    'fueltype_consideration_4_6',
    'fueltype_consideration_5_6',
    'fueltype_consideration_6_6',
    'fueltype_consideration_7_6',
    'fueltype_consideration_8_6',
    'fueltype_consideration_9_6',
    'fueltype_consideration_10_6',
    'fueltype_consideration_11_6',
    'fueltype_consideration_12_6',
    'fueltype_consideration_13_6',
    'fueltype_consideration_1_7',
    'fueltype_consideration_2_7',
    'fueltype_consideration_3_7',
    'fueltype_consideration_4_7',
    'fueltype_consideration_5_7',
    'fueltype_consideration_6_7',
    'fueltype_consideration_7_7',
    'fueltype_consideration_8_7',
    'fueltype_consideration_9_7',
    'fueltype_consideration_10_7',
    'fueltype_consideration_11_7',
    'fueltype_consideration_12_7',
    'fueltype_consideration_13_7',
    'fueltype_consideration_1_8',
    'fueltype_consideration_2_8',
    'fueltype_consideration_3_8',
    'fueltype_consideration_4_8',
    'fueltype_consideration_5_8',
    'fueltype_consideration_6_8',
    'fueltype_consideration_7_8',
    'fueltype_consideration_8_8',
    'fueltype_consideration_9_8',
    'fueltype_consideration_10_8',
    'fueltype_consideration_11_8',
    'fueltype_consideration_12_8',
    'fueltype_consideration_13_8',
    'consideration_1',
    'consideration_2',
    'consideration_3',
    'consideration_4',
    'consideration_5',
    'consideration_6',
    'consideration_7',
    'consideration_8',
    'consideration_9',
    'consideration_10',
    'consideration_11',
    'consideration_12',
    'consideration_13',
    'consideration_14',
    'consideration_15',
    'consideration_16',
    'consideration_17',
    'consideration_18',
    'consideration_19',
    'consideration_20',
    'consideration_21',
    'consideration_22',
    'consideration_23',
    'consideration_24',
    'consideration_25',
    'consideration_26',
    'consideration_27',
    'consideration_28',
    'consideration_29',
    'consideration_30',
    'consideration_31',
    'consideration_32',
    'consideration_33',
    'consideration_34',
    'consideration_35',
    'consideration_36',
    'consideration_37',
    'consideration_38',
    'consideration_39',
    'consideration_40',
    'consideration_41',
    'consideration_42',
    'consideration_43',
    'consideration_44',
    'consideration_45',
    'consideration_46',
    'consideration_47',
    'consideration_48',
    'consideration_49',
    'consideration_50',
    'consideration_51',
    'consideration_52',
    'consideration_53',
    'consideration_54',
    'consideration_55',
    'consideration_56',
    'consideration_57',
    'consideration_58',
    'consideration_59',
    'consideration_60',
    'consideration_61',
    'consideration_62',
    'consideration_63',
    'consideration_64',
    'consideration_65',
    'consideration_66',
    'consideration_67',
    'consideration_68',
    'consideration_69',
    'consideration_70',
    'consideration_71',
    'consideration_72',
    'consideration_73',
    'consideration_74',
    'consideration_75',
    'consideration_76',
    'consideration_77',
    'consideration_78',
    'consideration_79',
    'consideration_80',
    'consideration_81',
    'consideration_82',
    'consideration_83',
    'consideration_84',
    'consideration_85',
    'consideration_86',
    'consideration_87',
    'consideration_88',
    'consideration_89',
    'consideration_90',
    'consideration_91',
    'consideration_92',
    'consideration_93',
    'consideration_94',
    'consideration_95',
    'consideration_96',
    'consideration_97',
    'consideration_98',
    'consideration_99',
    'consideration_100',
    'consideration_101',
    'consideration_102',
    'consideration_103',
    'consideration_104']
main_df.drop(columns=col_drop, inplace=True)


In [10]:
main_df.head()

,sampno,source,language,age_grp,california,county,region,future_decision_role,num_hh_vehicles,household_members_1,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,192850,4,1,3,1,37,4,1,2,0,...,2,2,3,3,1,1,,2,1,11
1,193449,4,1,3,1,34,6,3,3,0,...,3,2,2,2,3,1,,1,,8
2,193440,4,1,3,1,50,1,1,2,0,...,4,2,3,2,1,1,,2,2,11
3,193133,4,1,4,1,33,2,1,1,0,...,2,2,1,2,3,1,,2,2,7
4,193400,4,1,3,1,1,5,3,3,0,...,4,2,2,2,2,1,,2,2,7


In [11]:
main_df.dtypes

sampno            int64
source            int64
language          int64
age_grp           int64
california        int64
                  ...  
housing           int64
housing_1_8_x    object
solar             int64
solar_future     object
income            int64
Length: 83, dtype: object

In [12]:
main_df.nunique()

sampno           4248
source              6
language            2
age_grp             3
california          1
                 ... 
housing             8
housing_1_8_x      21
solar               2
solar_future        3
income             11
Length: 83, dtype: int64

In [13]:
# Driver Demographics table
print(person_df.shape)
person_df.head()

(8365, 29)


,sampno,perid,perno,relationship,gender,employment,student,education,license,vehno,...,school_distance,ethnicity,race_1,race_2,race_3,race_4,race_5,race_6,race_7,race_6_x
0,190001,190001.p01,1,8,1,4,4,6,1,190001.v01,...,,2,0,0,0,0,1,0,0,
1,190003,190003.p01,1,8,2,4,4,8,1,190003.v01,...,,2,0,0,0,0,1,0,0,
2,190002,190002.p01,1,8,2,4,4,1,2,,...,,2,0,0,0,0,1,0,0,
3,190005,190005.p01,1,8,1,4,4,6,1,190005.v02,...,,2,0,0,0,0,1,0,0,
4,190002,190002.p02,2,1,1,1,4,2,1,,...,,2,0,0,0,0,1,0,0,


In [14]:
person_df.dtypes

sampno              int64
perid              object
perno               int64
relationship        int64
gender              int64
employment          int64
student             int64
education           int64
license             int64
vehno              object
vehicle_string     object
drive_freq         object
transit_freq        int64
tnc_freq            int64
job_type           object
work_mode          object
work_distance      object
work_days          object
school_mode        object
school_distance    object
ethnicity           int64
race_1              int64
race_2              int64
race_3              int64
race_4              int64
race_5              int64
race_6              int64
race_7              int64
race_6_x           object
dtype: object

In [15]:
person_df.nunique()

sampno             4248
perid              8365
perno                 9
relationship          8
gender                4
employment            5
student               4
education             8
license               2
vehno              6679
vehicle_string        1
drive_freq            5
transit_freq         34
tnc_freq             21
job_type              5
work_mode            17
work_distance       123
work_days             9
school_mode          13
school_distance      59
ethnicity             3
race_1                2
race_2                2
race_3                2
race_4                2
race_5                2
race_6                2
race_7                2
race_6_x            127
dtype: int64

In [16]:
# EV Owner Vehicle & Purchase information
print(pev_df.shape)
pev_df.head()

(451, 113)


,sampno,vehno,year,make,model,fuel,veh_type,pev_miles_week,pev_miles_month,phev_percent,...,pev_incentives_2,pev_incentives_3,pev_incentives_4,pev_incentives_5,pev_incentives_6,pev_incentives_7,pev_experience,pev_recommend,pev_confidence,pev_confidence_4_x
0,190107,1,2017,FORD,REDACTED,3,3,250,,30,...,5,4,4,1,1,4,5,5,,
1,190111,3,2018,VOLKSWAGEN,NaN,5,1,175,,,...,5,5,5,1,1,4,5,4,,
2,190120,2,2016,NISSAN,NaN,5,2,160,,,...,4,4,4,2,1,2,7,5,,
3,190127,1,2013,TOYOTA,NaN,3,3,200,,85,...,7,7,7,2,7,5,7,4,,
4,190138,3,2017,CHEVROLET,NaN,5,2,200,,,...,5,5,5,1,1,1,7,5,2,


In [17]:
pev_df.dtypes

sampno                 int64
vehno                  int64
year                   int64
make                  object
model                 object
                       ...  
pev_incentives_7       int64
pev_experience         int64
pev_recommend          int64
pev_confidence        object
pev_confidence_4_x    object
Length: 113, dtype: object

In [18]:
pev_df.nunique()

sampno                451
vehno                   7
year                   10
make                   19
model                   1
                     ... 
pev_incentives_7        7
pev_experience          7
pev_recommend           5
pev_confidence          5
pev_confidence_4_x      3
Length: 113, dtype: int64

In [19]:
pev_df.groupby(pev_df['fuel']).count()

,sampno,vehno,year,make,model,veh_type,pev_miles_week,pev_miles_month,phev_percent,battery_miles,...,pev_incentives_2,pev_incentives_3,pev_incentives_4,pev_incentives_5,pev_incentives_6,pev_incentives_7,pev_experience,pev_recommend,pev_confidence,pev_confidence_4_x
fuel,,,,,,,,,,,,,,,,,,,,,
3,173,173,173,173,1,173,173,173,173,173,...,173,173,173,173,173,173,173,173,173,173
5,278,278,278,278,0,278,278,278,278,278,...,278,278,278,278,278,278,278,278,278,278


In [20]:
# Vehicle information table

print(vehicle_df.shape)
vehicle_df.head()

(8049, 46)


,sampno,vehno,vehicle_num,num_hh_vehicles,year,make,make_other,make_clean,model,model_other,...,personal_miles,business_miles,company_primary_driver,company_primary_driver_id,other_drivers,behavior_1,behavior_2,behavior_3,behavior_4,behavior_5
0,190001,1,1,1,1998,TOYOTA,,TOYOTA,REDACTED,REDACTED,...,,,,,,,,,,
1,190003,1,1,2,2015,LEXUS,,LEXUS,NaN,NaN,...,,,,,,,,,,
2,190005,2,2,2,2002,CHRYSLER,,CHRYSLER,NaN,NaN,...,,,,,,,,,,
3,190006,1,1,2,2018,TOYOTA,,TOYOTA,NaN,NaN,...,,,,,,,,,,
4,190009,3,3,3,2002,HONDA,,HONDA,NaN,NaN,...,,,,,,,,,,


In [21]:
vehicle_df.dtypes

sampno                        int64
vehno                         int64
vehicle_num                   int64
num_hh_vehicles               int64
year                          int64
make                         object
make_other                   object
make_clean                   object
model                        object
model_other                  object
model_clean                  object
fuel                          int64
fuel_clean                    int64
veh_type                      int64
veh_type_clean                int64
vehicle_status               object
flag_hev                      int64
flag_phev                     int64
flag_fcev                     int64
flag_bev                      int64
obtain                        int64
company_car                  object
current_acquired_year        object
current_acquired_month       object
annual_mileage               object
electric_percent             object
tnc_veh                      object
tnc_miles                   

In [22]:
vehicle_df.nunique()

sampno                       4136
vehno                           8
vehicle_num                     8
num_hh_vehicles                 8
year                           40
make                           53
make_other                     36
make_clean                     69
model                           1
model_other                     1
model_clean                     1
fuel                            8
fuel_clean                      8
veh_type                       13
veh_type_clean                 13
vehicle_status                  5
flag_hev                        2
flag_phev                       2
flag_fcev                       2
flag_bev                        2
obtain                          6
company_car                     3
current_acquired_year          40
current_acquired_month         13
annual_mileage                281
electric_percent               37
tnc_veh                         3
tnc_miles                      51
delivery                        3
delivery_miles

In [23]:
vehicle_df.groupby(vehicle_df['fuel_clean']).count()

,sampno,vehno,vehicle_num,num_hh_vehicles,year,make,make_other,make_clean,model,model_other,...,personal_miles,business_miles,company_primary_driver,company_primary_driver_id,other_drivers,behavior_1,behavior_2,behavior_3,behavior_4,behavior_5
fuel_clean,,,,,,,,,,,,,,,,,,,,,
1,6338,6338,6338,6338,6338,6338,6338,6338,1,1,...,6338,6338,6338,6338,6338,6338,6338,6338,6338,6338
2,587,587,587,587,587,587,587,587,0,0,...,587,587,587,587,587,587,587,587,587,587
3,220,220,220,220,220,220,220,220,0,0,...,220,220,220,220,220,220,220,220,220,220
4,118,118,118,118,118,118,118,118,0,0,...,118,118,118,118,118,118,118,118,118,118
5,333,333,333,333,333,333,333,333,0,0,...,333,333,333,333,333,333,333,333,333,333
6,306,306,306,306,306,306,306,306,0,0,...,306,306,306,306,306,306,306,306,306,306
8,140,140,140,140,140,140,140,140,0,0,...,140,140,140,140,140,140,140,140,140,140
9,7,7,7,7,7,7,7,7,0,0,...,7,7,7,7,7,7,7,7,7,7


## CONNECT TO POSTGRES AND UPDATE TABLES

In [29]:
# from config import password

param_dic={
    "host": "localhost",
    "database": "Energy_Analysis",
    "user" : "postgres",
    "password" : "password"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
FATAL:  password authentication failed for user "postgres"



NameError: name 'sys' is not defined

In [25]:
connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
    
def to_alchemy(df, table):
    """""
    Push dataframe to postgres table
    """
    engine = create_engine(connect)
    df.to_sql(
        table, 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
    print("to_sql update complete")

In [26]:
to_alchemy(main_df, 'tbl_main')

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
to_alchemy(dce_df, 'tbl_dce')

In [54]:
to_alchemy(person_df, 'tbl_person')

to_sql update complete


In [55]:
to_alchemy(pev_df, 'tbl_pev')

to_sql update complete


In [56]:
to_alchemy(vehicle_df, 'tbl_vehicle')

to_sql update complete
